In [833]:
import numpy as np
import math
import scipy.stats as stats
from scipy.optimize import minimize
import pandas as pd
import statsmodels.api as sm

# Problem 1
For problem 1, we use each of the 3 models of returns to generate the expected price with r_t ~ N(0, sigma^2). Then we calculate the mean and standard deviation of each of the  prices. 

In [834]:
# def price_generator(p_t0, sigma, n, method = "Classical Brownian Motion"):
#     price = np.zeros(n)
#     price[0] = p_t0
#     returns = np.random.normal(0, sigma, size = n)
#     if(method.upper() == "CLASSICAL BROWNIAN MOTION"):
#         for i in range(1, n):
#             price[i] = price[i - 1] + returns[i]
#     elif(method.upper() == "ARITHMETIC RETURN SYSTEM"):
#         for i in range(1, n):
#             price[i] = price[i - 1] * (returns[i] + 1)
#     else:
#         for i in range(1, n):
#             price[i] = price[i - 1] *  math.exp(returns[i])
#     return price

In [835]:
def price_generator(p_t0, sigma, t, n, method = "Classical Brownian Motion"):
    price_t = np.zeros(n)
    if(method.upper() == "CLASSICAL BROWNIAN MOTION"):
        for i in range(0, n):
            price_t[i] = p_t0
            returns = np.random.normal(0, sigma, size = t)
            for j in range(1, t):
                price_t[i] = price_t[i] + returns[j]
    elif(method.upper() == "ARITHMETIC RETURN SYSTEM"):
        for i in range(0, n):
            price_t[i] = p_t0
            returns = np.random.normal(0, sigma, size = t)
            for j in range(1, t):
                price_t[i] = price_t[i] * (returns[j] + 1)
    else:
        for i in range(0, n):
            price_t[i] = p_t0
            returns = np.random.normal(0, sigma, size = t)
            for j in range(1, t):
                price_t[i] = price_t[i] *  math.exp(returns[j])
    return price_t

In [836]:
def price_analyser(price, method):
    mean = price.mean()
    standard_deviation = price.std()
    print("Mean of " + method + "is", f"{mean:.3f}")
    print("Standard deviation of " + method + " is", f"{standard_deviation:.3f}")

In [837]:
method = "Classical Brownian Motion"
classical_brownian_motion_price = price_generator(100, 0.1, 10, 10000, method)
price_analyser(classical_brownian_motion_price, method)

Mean of Classical Brownian Motionis 100.001
Standard deviation of Classical Brownian Motion is 0.302


In [878]:
method = "Arithmetic Return System"
classical_brownian_motion_price = price_generator(100, 0.1, 10, 100000, method)
price_analyser(classical_brownian_motion_price, method)

Mean of Arithmetic Return Systemis 100.108
Standard deviation of Arithmetic Return System is 30.657


In [867]:
method = "Log Return"
classical_brownian_motion_price = price_generator(100, 0.1, 10, 100000, method)
price_analyser(classical_brownian_motion_price, method)

Mean of Log Returnis 104.679
Standard deviation of Log Return is 32.136


# Problem 2
* Implement the return_calculate() function with all of the three ways to calculation return
* Calculate the arithmetic returns of DailyPrice.csv and remove the mean from META
* Calculate the VaR
    * Using a nomal distribution
    * Using a normal distribution with an Exponentially Weighted variance(lambda = 0.94)
    * Using a MLE fitted T distribution
    * Using a fitted AR(1) model
    * Using a Historic Simulation

In [842]:
def return_calculate(prices, method="DISCRETE", dateColumn="Date"):
    vars_ = prices.columns
    nVars = len(vars_)
    vars_ = [var for var in vars_ if var != dateColumn]
    if nVars == len(vars_):
        raise ValueError(f"dateColumn: {dateColumn} not in DataFrame: {vars_}")
    nVars = nVars - 1
    p = prices[vars_].to_numpy()
    n, m = p.shape
    p2 = np.empty((n-1, m))
    if method.upper() == "DISCRETE" or method.upper() == "LOG":
    # Calculate returns
        for i in range(n - 1):
            for j in range(m):
                p2[i, j] = p[i + 1, j] / p[i, j]
        
        if method.upper() == "DISCRETE":
            p2 = p2 - 1.0
        else:
            p2 = np.log(p2)
    elif method.upper() == "CLASSIC":
        for i in range(n - 1):
            for j in range(m):
                p2[i, j] = p[i + 1, j] - p[i, j]
    else:
        raise ValueError(f"method: {method} must be in ('LOG', 'DISCRETE', 'CLASSIC')")
    dates = prices[dateColumn].iloc[1:n].to_numpy()
    out = pd.DataFrame({dateColumn: dates})
    for i in range(nVars):
        out[vars_[i]] = p2[:, i]
    return out

In [843]:
daily_prices = pd.read_csv("DailyPrices.csv")
daily_returns = return_calculate(daily_prices, method = "DISCRETE",dateColumn="Date")
# daily_returns

/var/folders/14/qnbmw8kx46x2ncgblvbvxh1m0000gn/T/ipykernel_21142/1937632565.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars_[i]] = p2[:, i]
/var/folders/14/qnbmw8kx46x2ncgblvbvxh1m0000gn/T/ipykernel_21142/1937632565.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars_[i]] = p2[:, i]


In [844]:
meta_return = daily_returns['META'].values
meta_return_mean = meta_return.mean()
normalize_meta_return =  meta_return - meta_return_mean

In [845]:
alpha = 0.05

Normal Distribution

In [846]:
sigma = normalize_meta_return.std()
normal_distribution_model = np.random.normal(0, sigma , 10000)
VaR_normal_distribution = -np.quantile(normal_distribution_model, alpha)

print("VaR for Normal Distribution model is", VaR_normal_distribution)


VaR for Normal Distribution model is 0.05451621865393625


Normal Distribution with Exponentially Weighted variance

In [847]:
l = 0.94
total_weight = 0
length = len(normalize_meta_return)
weights = np.zeros(length)
for i in range(length):
    weights[i] = (1 - l) * l ** (length - i - 1)
    total_weight += weights[i]
for i in range(length):
    weights[i] = weights[i] / total_weight

sigma_ew = np.sqrt((normalize_meta_return * weights).T @ normalize_meta_return)
nomal_distribution_with_exponentially_weighted_variance_model = np.random.normal(0, sigma_ew, 10000)
VaR_ew = -np.quantile(nomal_distribution_with_exponentially_weighted_variance_model, alpha)

print("VaR for normal distribution with exponentially weighted variance is:", VaR_ew)

VaR for normal distribution with exponentially weighted variance is: 0.029777311974946645


MLE fitted T distribution

In [848]:
def neg_log_likelihood(params, data):
    df, loc, scale = params
    log_likelihood = np.sum(stats.t.logpdf(data, df=df, loc=loc, scale=scale))
    return -log_likelihood

In [879]:
initial_params = [3, 0, 1]
result = minimize(neg_log_likelihood, initial_params, args=normalize_meta_return, method='Nelder-Mead')
stimated_df, estimated_loc, estimated_scale = result.x
MLE_fitted_T_model = stats.t.rvs(df=stimated_df, loc=estimated_loc, scale=estimated_scale, size=10000)
VaR_MLE = -np.quantile(MLE_fitted_T_model, alpha)
print("VaR for MLE fitted T distribution is:", VaR_MLE)

VaR for MLE fitted T distribution is: 0.044348003552887996


AR（1）Model

In [864]:
model = sm.tsa.ARIMA(normalize_meta_return, order=(1, 0, 0))
results = model.fit().params
n = 10000
esim = np.random.normal(0,results[2],n)
AR1_model = np.zeros(n)
for i in range(n):
    AR1_model[i]=results[0] + normalize_meta_return[-1] * results[1] + esim[i]
VaR_AR1 = -np.quantile(AR1_model,alpha)
print("VaR for AR(1) model is:", VaR_AR1)

VaR for AR(1) model is: 0.0013821452738234984


In [865]:
# Calculate VaR using historic simulation
historic_model = np.random.choice(normalize_meta_return,n)
VaR_historic = -np.quantile(historic_model, alpha)

print("VAR for Historic Simulation is: {}".format(VaR_historic))

VAR for Historic Simulation is: 0.04202773934104901


# Problem 3


In [854]:
portfolio = pd.read_csv("portfolio.csv")

In [824]:
def return_calculate_1(prices, method="DISCRETE", dateColumn="Date"):
    vars_ = prices.columns
    nVars = len(vars_)
    vars_ = [var for var in vars_ if var != dateColumn]
    if nVars == len(vars_):
        raise ValueError(f"dateColumn: {dateColumn} not in DataFrame: {vars_}")
    nVars = nVars - 1
    p = prices[vars_].to_numpy()
    n, m = p.shape
    p2 = np.empty((n-1, m))
    for i in range(n-1):
        for j in range(m):
            p2[i, j] = p[i+1, j] / p[i, j]
    if method.upper() == "DISCRETE":
        p2 = p2 - 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)
    else:
        raise ValueError(f"method: {method} must be in (\"LOG\",\"DISCRETE\")")
    dates = prices[dateColumn].iloc[1:n].to_numpy()
    out = pd.DataFrame({dateColumn: dates})
    for i in range(nVars):
        out[vars_[i]] = p2[:, i]
    return out

In [855]:
def get_portfolio_price(portfolio, prices, portfolio_name, Delta=False):
    if portfolio_name == "All":
        assets = portfolio.drop('Portfolio',axis=1)
        assets = assets.groupby(["Stock"], as_index=False)["Holding"].sum()
    else:
        assets = portfolio[portfolio["Portfolio"] == portfolio_name]     
    stock_codes = list(assets["Stock"])
    assets_prices = pd.concat([prices["Date"], prices[stock_codes]], axis=1) 
    # print(assets_prices)
    current_price = np.dot(prices[assets["Stock"]].tail(1), assets["Holding"])
    holdings = assets["Holding"]   
    if Delta == True:
        asset_values = assets["Holding"].values.reshape(-1, 1) * prices[assets["Stock"]].tail(1).T.values
        delta = asset_values / current_price       
        return current_price, assets_prices, delta   
    return current_price, assets_prices, holdings

In [856]:
def exp_weighted_cov(returns, lambda_=0.97):
    returns = returns.values
    mean_return = np.mean(returns, axis=0)
    normalized_returns = returns - mean_return

    n_timesteps = normalized_returns.shape[0]
    cov = np.cov(returns, rowvar=False)

    for t in range(1, n_timesteps):
        cov = lambda_ * cov + (1 - lambda_) * np.outer(normalized_returns[t], normalized_returns[t])
    return cov

In [857]:
# Calculate with Delta Normal
def calculate_delta_var(portfolio, prices, alpha=0.05, lambda_=0.94, portfolio_name="All"):
    current_price, assets_prices, delta = get_portfolio_price(portfolio, prices, portfolio_name, Delta=True)
    returns = return_calculate(assets_prices, dateColumn="Date").drop('Date', axis=1)
    assets_cov = exp_weighted_cov(returns, lambda_)
    p_sig = np.sqrt(np.transpose(delta) @ assets_cov @ delta)
    var_delta = -current_price * stats.norm.ppf(alpha) * p_sig
    return current_price[0], var_delta[0][0]

In [858]:
# Calculate with historical simulation
def calculate_historic_var(portfolio, prices, alpha=0.05,n_simulation=1000, portfolio_name="All"):
    current_price, assets_prices, holdings = get_portfolio_price(portfolio, prices, portfolio_name)  
    returns = return_calculate(assets_prices, dateColumn="Date").drop("Date", axis=1)  
    assets_prices = assets_prices.drop('Date',axis=1)
    sim_returns = returns.sample(n_simulation, replace=True)
    sim_prices = np.dot(sim_returns* assets_prices.tail(1).values.reshape(assets_prices.shape[1],),holdings)   
    var_hist = -np.percentile(sim_prices, alpha*100) 
    return current_price[0], var_hist, sim_prices

In [859]:
# Print the results
current_price, delta_var = calculate_delta_var(portfolio, daily_prices, portfolio_name='A')
current_price, hist_var, hist_sim_prices = calculate_historic_var(portfolio, daily_prices, portfolio_name='A')

print("The current value for A is: {:.2f}".format(current_price))
print("VaR for A using Delta Normal is: {:.2f}".format(delta_var))
print("VaR for A using Historic Simulation is: {:.2f}\n".format(hist_var))

The current value for A is: 1089316.16
VaR for A using Delta Normal is: 15426.97
VaR for A using Historic Simulation is: 17065.30



In [860]:
current_price, delta_var = calculate_delta_var(portfolio, daily_prices, portfolio_name='B')
current_price, hist_var, hist_sim_prices = calculate_historic_var(portfolio, daily_prices, portfolio_name='B')
print("The current value for B is: {:.2f}".format(current_price))
print("VaR for B using Delta Normal is: {:.2f}".format(delta_var))
print("VaR for B using Historic Simulation is: {:.2f}\n".format(hist_var))

The current value for B is: 574542.41
VaR for B using Delta Normal is: 8082.57
VaR for B using Historic Simulation is: 12115.66



In [861]:
current_price, delta_var = calculate_delta_var(portfolio, daily_prices, portfolio_name='C')
current_price, hist_var, hist_sim_prices = calculate_historic_var(portfolio, daily_prices, portfolio_name='C')
print("The current value for C is: {:.2f}".format(current_price))
print("VaR for C using Delta Normal is: {:.2f}".format(delta_var))
print("VaR for C using Historic Simulation is: {:.2f}\n".format(hist_var))

The current value for C is: 1387409.51
VaR for C using Delta Normal is: 18163.29
VaR for C using Historic Simulation is: 22187.13



In [862]:
current_price, delta_var = calculate_delta_var(portfolio, daily_prices, portfolio_name='All')
current_price, hist_var, hist_sim_prices = calculate_historic_var(portfolio, daily_prices, portfolio_name='All')
print("The current value for total is: {:.2f}".format(current_price))
print("VaR for total using Delta Normal is: {:.2f}".format(delta_var))
print("VaR for total using Historic Simulation is: {:.2f}\n".format(hist_var))

The current value for total is: 3051268.07
VaR for total using Delta Normal is: 38941.38
VaR for total using Historic Simulation is: 46871.90



/var/folders/14/qnbmw8kx46x2ncgblvbvxh1m0000gn/T/ipykernel_21142/1937632565.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars_[i]] = p2[:, i]
/var/folders/14/qnbmw8kx46x2ncgblvbvxh1m0000gn/T/ipykernel_21142/1937632565.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars_[i]] = p2[:, i]
